In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('clean_df.csv')

In [3]:
# pisahkan data train dan test
X = df.drop(columns='harga', axis=1)
y = df['harga']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

In [4]:
train.head()

In [5]:
train.shape

In [6]:
train.info()

In [7]:
train.describe()

## Exploratory Data Analysis (EDA)

In [8]:
# jumlah kamar tidur
f = plt.figure(figsize=(15,5))

f.add_subplot(1,2,1)
sns.countplot(data=train, x='kamar_tidur')

f.add_subplot(1,2,2)
plt.boxplot(train["kamar_tidur"])
plt.show()

- kamar paling banyak berjumlah 2-3
- terdapat rumah yang memiliki 32 kamar

In [9]:
# kamar tidur vs harga
fig = px.scatter(train, x="kamar_tidur", y="harga",
                 title="Korelasi Antara Jumlah Kamar Tidur & Harga Rumah",
                 color="harga"
                )
fig.show()
# import plotly.graph_objects as go
# fig_widget = go.FigureWidget(fig)
# fig_widget

- rumah dengan jumlah kamar yang sama memiliki harga bervariasi
- terdapat outlier dimana jumlah kamar cukup banyak dengan harga yang sangat rendah

In [10]:
# periksa outlier
train[train['kamar_tidur'] > 20]

In [11]:
# bandingkan kamar tidur dengan luas bangunan
fig = px.scatter(train, x="kamar_tidur", y="luas_bangunan",
                 title="Korelasi Antara Jumlah Kamar Tidur & Luas Bangunan",
                 color="luas_bangunan"
                 )
fig.show()

Dapat dilihat, rumah dengan kamar sejumlah 32 memiliki luas bangunan yang cukup kecil. Rumah dengan 70 m^2 tidak mungkin memiliki jumlah kamar sebanyak 32. 
Dapat disimpulkan bahwa observasi ini merupakan kesalahan input atau kesalahan pada tahap scraping, sehingga akan dihapus

In [12]:
# kamar mandi
f = plt.figure(figsize=(15,5))

f.add_subplot(1,2,1)
sns.countplot(data=train, x='kamar_mandi')

f.add_subplot(1,2,2)
plt.boxplot(train["kamar_mandi"])
plt.show()

In [13]:
fig = px.scatter(train, x="kamar_mandi", y="harga",
                 title="Korelasi Antara Jumlah Kamar Mandi & Harga Rumah",
                color="harga"
                )
fig.show()

- rumah dengan jumlah kamar mandi yang sama memiliki harga yang bervariasi
- terdapat outlier dimana jumlah kamar mandi cukup banyak dengan harga yang sangat rendah dibanding rumah lain

In [14]:
train[train['kamar_mandi'] > 10]

rumah dengan kamar mandi > 10 memiliki jumlah kamar mandi yang sama dengan kamar tidur

In [15]:
fig = px.scatter(train, x="kamar_mandi", y="luas_bangunan",
                 title="Korelasi Antara Jumlah Kamar Mandi & Luas Bangunan",
                color="luas_bangunan"
                )
fig.show()

In [16]:
# luas bangunan
# sebaran luas bangunan
f = plt.figure(figsize=(15,5))

f.add_subplot(1,2,1)
sns.kdeplot(train['luas_bangunan'])

f.add_subplot(1,2,2)
plt.boxplot(train["luas_bangunan"])
plt.show()

- mayoritas rumah memiliki luas bangunan di bawah 500 m2
- terdapat cukup banyak outlier

In [17]:
# luas bangunan vs harga 
fig = px.scatter(train, x="luas_bangunan", y="harga",
                 title="Korelasi Antara Jumlah Luas Bangunan & Harga Rumah",
                color="harga"
                )
fig.show()

In [18]:
# luas lahan
# sebaran luas lahan
f = plt.figure(figsize=(15,5))

f.add_subplot(1,2,1)
sns.kdeplot(train['luas_lahan'])

f.add_subplot(1,2,2)
plt.boxplot(train["luas_lahan"])
plt.show()

In [19]:
# luas lahan vs harga 
fig = px.scatter(train, x="luas_lahan", y="harga",
                 title="Korelasi Antara Jumlah Luas Lahan & Harga Rumah",
                color="harga"
                )
fig.show()

- terdapat outlier dimana terdapat luas lahan yang sangat luas dengan harga yang sangat murah apabila dibanding yang lainnya

Berdasarkan analisis awal, terdapat outlier pada semua feature numeric yang dapat mempengaruhi hasil model. Langkah selanjutnya adalah memberikan tiga perlakuan pada dataset, yaitu membiarkan outlier, menghapus outlier dengan Z-score, dan hapus outlier dengan IQR untuk kemudian dibandingkan hasil metric nya

In [20]:
# daerah dengan harga rumah termahal
loc_mean = train.groupby("lokasi")["harga"].mean().reset_index().sort_values(by='harga', ascending=True)
loc_mean

In [21]:
plt.figure(figsize=(8, 6))
plt.barh(loc_mean['lokasi'], loc_mean['harga'])
plt.xlabel('Rata-Rata Harga Rumah (Juta)')
plt.ylabel('Lokasi')
plt.title('Rata-Rata Harga Rumah Berdasarkan Lokasi\n')
plt.grid(axis='x')
labels, locations = plt.xticks()
plt.xticks(labels, (labels/1000000).astype(int))
plt.tight_layout()
plt.show()

Dapat dilihat dari bar chart di atas, daerah Cinere merupakan daerah yang mempunyai rata-rata harga rumah termahal di kota Depok. Diikuti oleh Limo, Beji, dsb

In [22]:
# save hasil split train dan test
train.to_csv("train_df.csv", index=False)
test.to_csv("test_df.csv", index=False)

## Treat Outlier
Save hasil treat outlier untuk dibandingkan dengan membiarkan outlier

### Hapus semua outlier (Z-score) == train_zs

In [23]:
def zscore_outlier(df, column_name, threshold=3):
    # Calculate the z-score for the specified column
    z_scores = (df[column_name] - df[column_name].mean()) / df[column_name].std()

    # Find the indices of outliers based on the threshold
    outlier_indices = z_scores.abs() > threshold

    # Print the indices of outliers
    print("Indices of Outliers:")
    print(df[outlier_indices].index.tolist())

In [24]:
zscore_outlier(train, 'kamar_tidur')

In [25]:
train_zs = train.drop([111, 118, 288, 398, 422, 438, 468, 519, 603, 612, 618, 733, 748, 817, 993, 1024, 1051, 1082, 1199, 1227, 1355, 1552, 1579])
train_zs = train_zs.reset_index(drop=True)

In [26]:
zscore_outlier(train_zs, 'kamar_mandi')

In [27]:
train_zs = train_zs.drop([118, 141, 288, 422, 438, 468, 519, 580, 603, 618, 733, 817, 1024, 1227, 1355, 1462, 1579, 1686])
train_zs = train_zs.reset_index(drop=True)

In [28]:
zscore_outlier(train_zs, 'luas_bangunan')

In [29]:
train_zs = train_zs.drop([118, 141, 165, 282, 398, 520, 580, 603, 618, 733, 748, 783, 817, 959, 993, 1000, 1015, 1024, 1051, 1060, 1124, 1286, 1351, 1388, 1435, 1579, 1686])
train_zs = train_zs.reset_index(drop=True)

In [30]:
zscore_outlier(train_zs, 'luas_lahan')

In [31]:
train_zs = train_zs.drop([996])
train_zs = train_zs.reset_index(drop=True)

In [32]:
train_zs.shape

In [33]:
train_zs.to_csv("train_zscore_outlier.csv", index=False)

### Hapus semua outlier (IQR) == train_iqr

In [34]:
def iqr_outlier(df, column_name):
    # Get the column data
    column_data = df[column_name]

    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = column_data.quantile(0.25)
    Q3 = column_data.quantile(0.75)

    # Calculate the IQR (Interquartile Range)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find the indices of outliers
    outlier_indices = (column_data < lower_bound) | (column_data > upper_bound)

    # Print the indices of outliers
    print("Indices of Outliers:")
    print(df[outlier_indices].index.tolist())

In [35]:
iqr_outlier(train, 'kamar_tidur')

In [36]:
idx = [13, 94, 111, 118, 141, 187, 237, 251, 252, 282, 288, 352, 398, 405, 422, 438, 446, 468, 507, 511, 519, 520, 523, 525, 580, 596, 603, 612, 618, 650, 664, 722, 728, 733, 742, 748, 817, 825, 838, 865, 867, 890, 919, 938, 951, 959, 979, 993, 1000, 1015, 1024, 1051, 1082, 1158, 1164, 1173, 1199, 1200, 1206, 1227, 1241, 1257, 1264, 1342, 1345, 1354, 1355, 1388, 1462, 1504, 1517, 1518, 1519, 1523, 1524, 1552, 1572, 1579, 1686, 1703]
train_iqr = train.drop(idx).reset_index(drop=True)

In [37]:
iqr_outlier(train_iqr, 'kamar_mandi')

In [38]:
idx = [288, 438, 519, 603, 733, 817, 1024, 1227, 1355]
train_iqr = train_iqr.drop(idx).reset_index(drop=True)

In [39]:
iqr_outlier(train_iqr, 'luas_bangunan')

In [40]:
idx = [30, 51, 74, 82, 123, 126, 133, 160, 185, 233, 241, 259, 277, 292, 362, 376, 384, 430, 432, 462, 473, 475, 481, 496, 505, 553, 567, 578, 580, 581, 595, 596, 606, 660, 684, 685, 694, 695, 698, 706, 742, 746, 754, 779, 781, 791, 806, 869, 876, 908, 914, 932, 964, 976, 1002, 1016, 1018, 1035, 1063, 1064, 1091, 1104, 1110, 1114, 1160, 1188, 1216, 1226, 1276, 1278, 1279, 1349, 1355, 1358, 1408, 1432, 1459, 1555, 1575, 1616, 1645]
train_iqr = train_iqr.drop(idx).reset_index(drop=True)

In [41]:
iqr_outlier(train_iqr, 'luas_lahan')

In [42]:
idx = [0, 62, 64, 120, 128, 133, 143, 207, 245, 272, 277, 312, 319, 329, 336, 372, 392, 401, 407, 414, 433, 447, 448, 461, 486, 513, 518, 520, 543, 544, 545, 566, 602, 616, 622, 628, 644, 649, 655, 668, 673, 694, 698, 702, 712, 737, 755, 792, 796, 807, 813, 838, 867, 901, 915, 917, 930, 973, 979, 984, 1015, 1028, 1030, 1034, 1048, 1054, 1069, 1072, 1157, 1185, 1186, 1189, 1204, 1218, 1221, 1223, 1236, 1239, 1247, 1272, 1286, 1296, 1324, 1362, 1366, 1371, 1391, 1394, 1411, 1416, 1421, 1425, 1443, 1469, 1484, 1504, 1529, 1536, 1552]
train_iqr = train_iqr.drop(idx).reset_index(drop=True)

In [43]:
train_iqr.shape

In [44]:
train_iqr.to_csv("train_iqr_outlier.csv", index=False)